# DB2-Salesforce connector: Citation information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Record_ID__c'
object_id = 'nanoHUB_citations__c'

# login parameters to be handled by Papermill
# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
import os
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [3]:
# Basic tool information
sql_query = "select *, c.id as c_id from jos_citations as c \
            left join jos_citations_types as t \
            on c.type = t.id \
            where uid=2143"

# display
print(sql_query)

select *, c.id as c_id from jos_citations as c             left join jos_citations_types as t             on c.type = t.id             where uid=2143


In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

In [5]:
# convert and clean some columns, so we can, for example, sort values
df['date_publish'] = pd.to_datetime(df['date_publish'], errors='coerce') 
df['created'] = pd.to_datetime(df['created'], errors='coerce') 
df['date_submit'] = pd.to_datetime(df['date_submit'], errors='coerce') 
df['date_accept'] = pd.to_datetime(df['date_accept'], errors='coerce') 

In [6]:
# display
display(df.head(1))
display('size='+str(df.shape))

,id,uid,affiliated,fundedby,created,address,author,booktitle,chapter,cite,...,custom2,custom3,custom4,pdf_url,id,type,type_title,type_desc,type_export,c_id
0,2035,2143,1,-1,NaT,,Prasad Sarangapani{{72949}};Daniel Mejia{{5234...,,,,...,None,None,None,https://nanohub.org/nanoHUBCitations/citmanage...,2,journal,Journal,None,None,2035


'size=(2507, 68)'

## Match data with Salesforce format

In [7]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Record_ID__c']      = df['c_id']
df_sf['URL__c']            = df['url']
df_sf['NCN_Affiliated__c'] = df['affiliated'].apply(lambda x: 1 if x==1 else 0)
df_sf['Author_List__c']    = df['author']
df_sf['Journal__c']        = df['journal']
df_sf['Year__c']           = df['year']
df_sf['Type__c']           = df['ref_type']
df_sf['Notes__c']          = df['notes']
df_sf['Name']              = df['title'].apply(lambda x: x[:70])
df_sf['Full_title__c']     = df['title']
df_sf['Format__c']         = df['type_title']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Record_ID__c,2035,2730
URL__c,http://scholar.google.com/scholar?hl=en&q=Quan...,http://scholar.google.com/scholar?hl=en&q=Unde...
NCN_Affiliated__c,1,0
Author_List__c,Prasad Sarangapani{{72949}};Daniel Mejia{{5234...,Muhammad Usman{{9702}};Christopher Broderick{{...
Journal__c,,aXv.org cond-mat.mtlr-sci
Year__c,2015,2017
Type__c,R,R
Notes__c,{{IEEE}}\r\nDon't know what publication.,
Name,Quantum Dot Lab: An Online Platform For Quantu...,Understanding The Impact Of Disorder Effects I...
Full_title__c,Quantum Dot Lab: An Online Platform For Quantu...,Understanding The Impact Of Disorder Effects I...


## To Salesforce Sales Cloud CRM

In [8]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [9]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmIssAAF
hello


[Success] CSV upload successful. Job ID = 7505w00000TmIssAAF


[Success] Closing job successful. Job ID = 7505w00000TmIssAAF


In [10]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000TmIssAAF',
 'operation': 'upsert',
 'object': 'nanoHUB_citations__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-21T06:30:44.000+0000',
 'systemModstamp': '2021-01-21T06:30:45.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Record_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [11]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
